In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
data1 = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
data2 = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
data3 = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
data1.head()
data2.head()
data3.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,1e896ff4-117b-4484-ac1d-fe3e753a4914,23f7e309-a99d-41a6-afa3-a05e33ba5432,b7a02932-0ce0-4769-9a3b-e4a398838ddf,b033d198-4984-4370-aa0c-c3d08399bde9
1,6d6c7236-649e-4b05-99e9-ddb085891243,03690412-d735-442b-bd09-8e55333c4ff1,853b4ab4-d45d-4fec-93cd-b74b044bb43a,6fa85722-896b-4bda-a8d3-dc70e82c7b90
2,b69ecd8b-a0cf-45fe-9916-3fe400c81cc9,74afaf8f-0e27-4e69-8a07-7795b4d2dca5,0a83494b-7db2-4f9d-9d77-58ec9cdb8c5f,ce741b3d-d137-4744-8ec4-4f00f9b954f7
3,e6cd18fa-dd5f-4cf5-b45c-d30d9c957085,95a4ee4c-f65d-422b-be57-ee954a9e19f3,918faa93-f824-4c15-8106-0a75a421f132,ff6fa94b-9d51-4b3f-b252-f1d7b0018c84
4,ad574736-b69c-44d9-9848-7902615012f5,20487871-ae5b-42f4-a1ed-ddd467748d6d,24b778a8-b330-4434-b505-3d9946db125c,079ebc49-6005-459b-9446-187d53443263


In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    merged_data_url: Path
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            merged_data_url = config.merged_data_url,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.hybrid_recommender import logger


In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.merged_data_url)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-30 16:29:00,940: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-30 16:29:00,942: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-30 16:29:00,944: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-30 16:29:00,945: INFO: common: created directory at: artifacts]
[2025-06-30 16:29:00,945: INFO: common: created directory at: artifacts/data_validation]
